In [1]:
using Pkg, REopt, HiGHS, JSON, JuMP, CSV, DataFrames, Sockets, Formatting

# Replace with your API Key
ENV["NREL_DEVELOPER_API_KEY"] = "EMXhDJ1wZrilRcOmFMHNIOASzWog8RLtJIPr22ep"
Pkg.status()

Precompiling REopt
  ? ArchGDAL
        Info Given REopt was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
  ? REopt
[ Info: Precompiling REopt [d36ad4e8-d74a-4f7a-ace1-eaea049febf6]
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
[ Info: Skipping precompilation since __precompile__(false). Importing REopt [d36ad4e8-d74a-4f7a-ace1-eaea049febf6].
Precompiling ArchGDAL
        Info Given ArchGDAL was explicitly requested, output will be shown live 
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of precompilation.
  ? ArchGDAL
[ Info: Precompiling ArchGDAL [c9ce4bd3-c3d5-55b8-8973-c0e20141b8c3]
ERROR: Method overwriting is not permitted during Module precompilation. Use `__precompile__(false)` to opt-out of prec

Status `C:\Users\xli1\Documents\REopt\FY24\TLDRD\REopt-MPC\Project.toml`
  [336ed68f] CSV v0.10.15
  [a93c6f00] DataFrames v1.8.1
  [59287772] Formatting v0.4.3
  [87dc4568] HiGHS v1.20.1
⌅ [682c06a0] JSON v0.21.4
  [4076af6c] JuMP v1.29.4
  [d36ad4e8] REopt v0.52.0 `https://github.com/NREL/REopt.jl.git#high-temp-thermal-storage`
  [6462fe0b] Sockets
Info Packages marked with ⌅ have new versions available but compatibility constraints restrict them from upgrading. To see why use `status --outdated`


In [2]:
# Paths
inputs_path = "inputs"
outputs_path = "outputs"

# Filenames
load_file = "Hospital_10MW.csv"
TMY_prod_factors_file = "TMY_production_factors.csv" 
pv_2022_prod_factors_file = "pv_2022_production_factor.csv" 
energy_cost_file = "energy_cost_tou_2022.csv"
demand_cost_file = "demand_cost_tou_2022.csv"
emissions_file = "hourly_moer_2022.csv"

# Load relevant csvs 
full_load = CSV.read(inputs_path * "/" * load_file, DataFrame)
TMY_prod_factors = CSV.read(inputs_path * "/" * TMY_prod_factors_file, DataFrame)
pv_2022_prod_factor = CSV.read(inputs_path * "/" * pv_2022_prod_factors_file, DataFrame)
energy_cost = CSV.read(inputs_path * "/" * energy_cost_file, DataFrame)
demand_cost = CSV.read(inputs_path * "/" * demand_cost_file, DataFrame)
hourly_emissions = CSV.read(inputs_path * "/" * emissions_file, DataFrame)

load = full_load[!,"loads_kW"]
energy_rate = energy_cost[!,"usd_per_kwh"]
demand_cost = demand_cost[!,"usd_per_kw"]
emissions_lbs_per_kwh = hourly_emissions[!,"MOER [lbsCO2/kWh]"]
month_transition_timesteps = [744, 1416, 2160, 2880, 3624, 4344, 5088, 5832, 6552, 7296, 8016, 8760]

# SET THESE! #####################################################
looping_method = "imperfect_1" # Options: perfect, imperfect_1

save_results = true
filename = "aries_scenario1" 

reopt_resource_type = "actual_year" # Options: TMY, actual_year, 8760 (strings)
control_algorithm_resource_type = "actual_year" # Options: TMY, actual_year
new_year_starting_ts = 25

allow_export = false
include_climate_in_objective = false
###################################################################

# System sizes
pv_kw = 21468 
bess_kw = 8930 
bess_kwh = 50219 

# Storage efficiencies
soc_min_fraction = 0.2
internal_efficiency_fraction = 0.975
inverter_efficiency_fraction = 0.96
rectifier_efficiency_fraction = 0.96

charge_efficiency = rectifier_efficiency_fraction * internal_efficiency_fraction^0.5
discharge_efficiency = inverter_efficiency_fraction * internal_efficiency_fraction^0.5

if reopt_resource_type == "TMY"
    pv_prod_factor = TMY_prod_factors[!,"pv_prod_factor"]
elseif reopt_resource_type == "actual_year"
    pv_prod_factor = pv_2022_prod_factor[!,"pv_2022"]
elseif reopt_resource_type == "8760"
    pv_prod_factor_forecast = pv_2022_prod_factor[!,"pv_2022"]
    pv_prod_factor_TMY = TMY_prod_factors[!,"pv_prod_factor"]
else
    print("Invalid reopt_resource_type")   
end

if control_algorithm_resource_type == "TMY"
    realized_pv_prod_factor = TMY_prod_factors[!,"pv_prod_factor"] #pv_2022_prod_factor[!,"pv_2022"]
elseif control_algorithm_resource_type == "actual_year"
    realized_pv_prod_factor = pv_2022_prod_factor[!,"pv_2022"]
else
    print("Invalid control_algorithm_resource_type")
end

# Demand cost inputs structure for initial post
tou_demand_rates = [10.0, 22.0, 18.0, 26.0]
tou_demand_time_steps = [[], []]
tou_previous_peak_demands = [0, 0]; 

rate_scenario = "tou"
heuristics_dispatch = false;

In [3]:
# UDP configuration parameters
const UDP_IP = "127.0.0.1"   # Must match listener IP
const UDP_PORT = 5005        # Must match listener port

function write_be_float64(io::IO, x::Float64)
    u = reinterpret(UInt64, x)
    write(io, hton(u))
end

write_be_float64 (generic function with 1 method)

In [4]:
post = Dict([
    ("Settings", Dict([
        ("include_climate_in_objective", include_climate_in_objective),
        ])
    ),
    ("Site", Dict([
        ("include_exported_elec_emissions_in_total", true),
        ])
    ),
    ("ElectricLoad", Dict([
        ("loads_kw", [])
        ])),
    ("ElectricTariff",Dict([ 
       ("energy_rates", []),
       ("tou_demand_rates", tou_demand_rates),
       ("tou_demand_time_steps", tou_demand_time_steps),
       ("tou_previous_peak_demands", tou_previous_peak_demands) 
        ])), 
    ("ElectricUtility", Dict([
        ("emissions_factor_series_lb_CO2_per_kwh", [0.0]),
        ("emissions_factor_series_lb_NOx_per_kwh", [0.0]),
        ("emissions_factor_series_lb_SO2_per_kwh", [0.0]),
        ("emissions_factor_series_lb_PM25_per_kwh", [0.0])
        ])
    ),
#     ("Financial", Dict([
#         ("CO2_cost_per_tonne", 100.0)
#         ]
#         )),
    ("PV", Dict([
        ("size_kw", pv_kw),
        ("production_factor_series", [])
        ]
        )), 
    ("ElectricStorage",Dict([
        ("size_kw", bess_kw), 
        ("size_kwh", bess_kwh), 
        ("charge_efficiency", charge_efficiency),
        ("discharge_efficiency", discharge_efficiency),
        ("soc_init_fraction", 0.2),
        ("soc_min_fraction", soc_min_fraction),
        ("daily_leakage_fraction", 0.0),
        ("fixed_dispatch_series", nothing)
        ]))
    ]);

In [5]:
function get_tou_demand_time_steps(cost_array, ts_array)
    tou_demand_ts = []
    for cost in cost_array
        push!(tou_demand_ts, findall(x->x==cost, ts_array))
    end
    return tou_demand_ts
end

get_tou_demand_time_steps (generic function with 1 method)

In [6]:
function dispatch_method_1(result, r1_ts, r2_ts, bess_kwh, previous_soc, charge_efficiency, discharge_efficiency, hours_per_time_step)

    # Process control execution
    load_ts = result["ElectricLoad"]["load_series_kw"][1]
    if r2_ts == 0
        bess_dispatch_ts = result["ElectricStorage"]["storage_to_load_series_kw"][1] - 
                           result["PV"]["electric_to_storage_series_kw"][1] -  
                           result["ElectricUtility"]["electric_to_storage_series_kw"][1]
    else
        bess_dispatch_ts = result["ElectricStorage"]["storage_to_load_series_kw"][1] - 
                           result["PV"]["electric_to_storage_series_kw"][1] - 
                           result["Wind"]["electric_to_storage_series_kw"][1] - 
                           result["ElectricUtility"]["electric_to_storage_series_kw"][1]
    end
    
    # RE resources are greater than the load (resource 1 is prioritized)
    if (r1_ts + r2_ts) >= load_ts
        util_to_load = 0
        bess_to_load = 0

        # Resource 1 generates enough to serve load by itself
        if r1_ts >= load_ts
            r1_to_load = load_ts
            r1_excess = r1_ts - r1_to_load
            r2_to_load = 0
            r2_excess = r2_ts

        # Both resources are need to serve load
        else 
            r1_to_load = r1_ts
            r1_excess = 0
            r2_to_load = load_ts - r1_to_load
            r2_excess = r2_ts - r2_to_load
        end

        # Control signal - Battery discharge
        if bess_dispatch_ts >= 0
            r1_to_bess = 0
            r2_to_bess = 0
            util_to_bess = 0
            bess_soc = previous_soc

         # Control signal - Battery charge
        else
            bess_soc = min(1.0, previous_soc + (abs(bess_dispatch_ts) * charge_efficiency * hours_per_time_step) / bess_kwh)

            # Resource 1 generates enough to charge battery after serving load
            if r1_excess > abs(bess_dispatch_ts)
                r1_to_bess = abs(bess_dispatch_ts)
                r1_excess = r1_excess - abs(bess_dispatch_ts)
                r2_to_bess = 0
                util_to_bess = 0 

            # Resource 1 and 2 together generate enough to charge battery after serving load
            elseif (r1_excess + r2_excess) > abs(bess_dispatch_ts)
                r1_to_bess = r1_excess
                r1_excess = 0
                r2_to_bess = abs(bess_dispatch_ts) - r1_to_bess
                r2_excess = r2_excess - r2_to_bess
                util_to_bess = 0

            # Excess RE is insufficient to charge battery    
            else 
                r1_to_bess = r1_excess
                r1_excess = 0
                r2_to_bess = r2_excess
                r2_excess = 0
                util_to_bess = abs(bess_dispatch_ts) - r1_to_bess - r2_to_bess
            end
        end

    # RE resouces are not enough to meet load
    else 
        r1_to_load = r1_ts
        r1_excess = 0
        r1_to_bess = 0
        r2_to_load = r2_ts
        r2_excess = 0            
        r2_to_bess = 0
        unmet_load = load_ts - r1_ts - r2_ts

        # Control signal - Battery discharge
        if bess_dispatch_ts >= 0
            util_to_bess = 0

            # Optimal battery discharge signal can meet the load
            if bess_dispatch_ts >= unmet_load
                bess_to_load = unmet_load
                util_to_load = 0
                bess_soc = max(soc_min_fraction, previous_soc - ((unmet_load/discharge_efficiency) * hours_per_time_step) / bess_kwh)

            # Optimal battery dispatch signal cannot meet the load
            else 
                bess_to_load = bess_dispatch_ts
                util_to_load = unmet_load - bess_dispatch_ts
                bess_soc = max(soc_min_fraction, previous_soc - ((bess_dispatch_ts/discharge_efficiency) * hours_per_time_step) / bess_kwh)
            end

        # Control signal - Battery charge  
        else 
            util_to_load = unmet_load
            util_to_bess = abs(bess_dispatch_ts)
            bess_to_load = 0
            bess_soc = min(1.0, previous_soc + (abs(bess_dispatch_ts) * charge_efficiency * hours_per_time_step) / bess_kwh)
        end
    end

    return r1_to_load, r1_to_bess, r1_excess, r2_to_load, r2_to_bess, r2_excess, bess_to_load, bess_soc, util_to_load, util_to_bess
end

dispatch_method_1 (generic function with 1 method)

In [7]:
# MPC Loop

hours_per_time_step = 1
horizon = 24
stopping_ts = 8760 # 8760 for full year run
starting_ts = 1
length_of_data = 8760

# Running sum variables
energy_cost_kwh = 0
soc_init_fraction = 0.2
tou_previous_peak_demands = zeros(length(tou_demand_rates))
monthly_demand_cost = []

tou_peaks_by_month = []

# Full results from each MPC loop
results = []

# Executed dispatch results 
dispatch_results = Dict([
    ("PV", Dict([
        ("electric_to_load_series_kw", []),
        ("electric_to_storage_series_kw", []),
        ("electric_to_grid_series_kw", []),
        ("electric_curtailed_series_kw", [])
        ])
    ),
    ("ElectricStorage", Dict([
        ("storage_to_load_series_kw", []),
        ("soc_series_fraction", [])
        ])
    ),        
    ("ElectricUtility", Dict([
        ("electric_to_load_series_kw", []),
        ("electric_to_storage_series_kw", []),
        ("emissions_series_tonnes_CO2", [])           
        ])
    ),
    ("ElectricLoad", Dict([
        ("load_series_kw", []),
        ])
    )
])

cost_series = []

sock = UDPSocket()
run_count = 0

for idx in range(starting_ts, stop=stopping_ts)
    
    end_ts = idx+horizon-1
    if end_ts > length_of_data
        end_ts = length_of_data
    end
    
    # Update forecast for each run
    if reopt_resource_type == "8760"        
        post["PV"]["production_factor_series"] = [pv_prod_factor_forecast[idx:end_ts]; 
                                                  pv_prod_factor_TMY[end_ts+1:length(pv_prod_factor_TMY)]; 
                                                  pv_prod_factor_TMY[1:idx-1]]
        
        splice_end_ts = new_year_starting_ts + idx - 2
        new_end_ts = 0
        if splice_end_ts > length_of_data
            new_end_ts = new_year_starting_ts + splice_end_ts - length_of_data - 1
            splice_end_ts = length_of_data
        end

        post["ElectricTariff"]["energy_rates"] = [energy_rate[idx:length(energy_rate)];
                                                  energy_rate[new_year_starting_ts:splice_end_ts];
                                                  energy_rate[new_year_starting_ts:new_end_ts]]
        post["ElectricLoad"]["loads_kw"] = [load[idx:length(load)];
                                            load[new_year_starting_ts:splice_end_ts];
                                            load[new_year_starting_ts:new_end_ts]]
        post["ElectricUtility"]["emissions_factor_series_lb_CO2_per_kwh"] = [emissions_lbs_per_kwh[idx:length(emissions_lbs_per_kwh)];
                                           emissions_lbs_per_kwh[new_year_starting_ts:splice_end_ts];
                                           emissions_lbs_per_kwh[new_year_starting_ts:new_end_ts]]
    else
        post["PV"]["production_factor_series"] = pv_prod_factor[idx:end_ts]
        
        post["ElectricTariff"]["energy_rates"] = energy_rate[idx:end_ts]
        post["ElectricLoad"]["loads_kw"] = load[idx:end_ts]
        post["ElectricUtility"]["emissions_factor_series_lb_CO2_per_kwh"] = emissions_lbs_per_kwh[idx:end_ts]
    end
    
    post["ElectricStorage"]["soc_init_fraction"] = soc_init_fraction
        
    if rate_scenario == "tou"
        tou_demand_time_steps = get_tou_demand_time_steps(tou_demand_rates, demand_cost[idx:end_ts])
        
#         post["ElectricTariff"]["tou_demand_rates"] = tou_demand_rates # Set earlier, doesn't change by loop
        post["ElectricTariff"]["tou_demand_time_steps"] = tou_demand_time_steps
        post["ElectricTariff"]["tou_previous_peak_demands"] = tou_previous_peak_demands
    end
    
    if heuristics_dispatch
        post["ElectricStorage"]["fixed_dispatch_series"] = bess_dispatch[idx:end_ts]
    end

    # Run optimization
    model = Model(optimizer_with_attributes(HiGHS.Optimizer, "output_flag" => false, "log_to_console" => false))
    result = run_mpc(model, post)
    
    # Save full set of results per MPC loop 
    push!(results, result) 
    
    # Get results needed for the next iteration of the MPC algorithm
    if looping_method == "perfect"
        
        r1_to_load = result["PV"]["electric_to_load_series_kw"][1]
        r1_to_bess = result["PV"]["electric_to_storage_series_kw"][1]
        # Not sure if REopt MPC ever sends anything to grid
        # Currently assumes all excess is exported or curtailed (export benefits not calculated though)
        r1_excess = result["PV"]["electric_to_grid_series_kw"][1] + result["PV"]["electric_curtailed_series_kw"][1]
        
        bess_to_load = result["ElectricStorage"]["storage_to_load_series_kw"][1]
        bess_soc = result["ElectricStorage"]["soc_series_fraction"][1]
        util_to_load = result["ElectricUtility"]["electric_to_load_series_kw"][1]
        util_to_bess = result["ElectricUtility"]["electric_to_storage_series_kw"][1]

        grid_power = max(result["ElectricUtility"]["electric_to_load_series_kw"][1] + 
                         result["ElectricUtility"]["electric_to_storage_series_kw"][1], 0)      
        
    elseif looping_method == "imperfect_1"
        
        r1_to_load, r1_to_bess, r1_excess, r2_to_load, r2_to_bess, r2_excess, bess_to_load, bess_soc, util_to_load, util_to_bess = dispatch_method_1(result, realized_pv_prod_factor[idx]*pv_kw, 0.0, bess_kwh, soc_init_fraction, charge_efficiency, discharge_efficiency, hours_per_time_step)
        grid_power = max(util_to_load + util_to_bess, 0)
         
    else
        print("Undefined Looping Method")
        break
    end
    
    try
        # 1. Pset (PV Setpoint):
        if allow_export
            Pset = r1_to_load + r1_to_bess + r1_excess
        else
            Pset = r1_to_load + r1_to_bess
        end

        # 2. B_set (Battery Setpoint):
        B_set = bess_to_load - (r1_to_bess + util_to_bess)

        # 3. Pack the two Float64 values as big-endian (>dd)
        io = IOBuffer()
        write_be_float64(io, Float64(Pset))
        write_be_float64(io, Float64(B_set))     
        message = take!(io)

        # 4. Send the message
        send(sock, IPv4(UDP_IP), UDP_PORT, message)

        run_count += 1
        println("Run $run_count: Pset = $(round(Pset, digits=4)), B_set = $(round(B_set, digits=4))")
        
#         sleep(1.0)
    catch e
        println("\nAn error occurred: $e")
    end
    
    # Process time series results for the timestep executed 
    push!(dispatch_results["PV"]["electric_to_load_series_kw"], r1_to_load)
    push!(dispatch_results["PV"]["electric_to_storage_series_kw"], r1_to_bess)
    if allow_export
        push!(dispatch_results["PV"]["electric_to_grid_series_kw"], r1_excess)
        push!(dispatch_results["PV"]["electric_curtailed_series_kw"], 0)
    else
        push!(dispatch_results["PV"]["electric_to_grid_series_kw"], 0)
        push!(dispatch_results["PV"]["electric_curtailed_series_kw"], r1_excess)
    end
    
    push!(dispatch_results["ElectricStorage"]["storage_to_load_series_kw"], bess_to_load)
    push!(dispatch_results["ElectricStorage"]["soc_series_fraction"], round(bess_soc, digits=6))

    push!(dispatch_results["ElectricUtility"]["electric_to_load_series_kw"], util_to_load)
    push!(dispatch_results["ElectricUtility"]["electric_to_storage_series_kw"], util_to_bess) 
    push!(dispatch_results["ElectricUtility"]["emissions_series_tonnes_CO2"], emissions_lbs_per_kwh[idx] * grid_power * hours_per_time_step) 
    
    push!(dispatch_results["ElectricLoad"]["load_series_kw"], result["ElectricLoad"]["load_series_kw"][1])
    
    # Calculate running sums of relevant values
    energy_cost_kwh += grid_power * post["ElectricTariff"]["energy_rates"][1]
    push!(cost_series, grid_power * post["ElectricTariff"]["energy_rates"][1])
    
    soc_init_fraction = bess_soc

    if rate_scenario == "tou"
        # Find which time of use period the current timestep is in
        i = findfirst(==(demand_cost[idx]), tou_demand_rates)
        tou_previous_peak_demands[i] = max(grid_power, tou_previous_peak_demands[i])
        
        # Calculate the total TOU demand charge at the end of each month
        # Currently, MPC returns demand charge = 0 if run for less than 1 month
        if idx in month_transition_timesteps
            append!(monthly_demand_cost, tou_previous_peak_demands .* tou_demand_rates)
            push!(tou_peaks_by_month, tou_previous_peak_demands)
            tou_previous_peak_demands = zeros(length(tou_demand_rates))
        end
    end
    
end

close(sock)
    
print("\n\nFinished loop\n")

[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.39 seconds.
[ Info: Results processing took 2.505 seconds.
[ Info: Results processing took 2.505 seconds.


Run 1: Pset = 0.0, B_set = 0.0
Run 2: Pset = 0.0, B_set = 0.0
Run 3: Pset = 0.0, B_set = 0.0
Run 4: Pset = 0.0, B_set = 0.0


[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.017 seconds.
[ Info: Results processing took 0.001 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.016 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.015 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Model built. Optimizing...


Run 5: Pset = 0.0, B_set = 0.0
Run 6: Pset = 0.0, B_set = 0.0
Run 7: Pset = 0.0, B_set = 0.0
Run 8: Pset = 3167.9254, B_set = 0.0

┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.015 seconds.
[ Info: Results processing took 0.001 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.015 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.016 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.016 seconds.
[ Info: Results processing took 0.001 seconds.
[ Info: Results processing took 0.002 seconds.


...

[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.011 seconds.
[ Info: Results processing took 0.001 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.013 seconds.
[ Info: Results processing took 0.001 seconds.
[ Info: Results processing took 0.001 seconds.
[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.014 seconds.
[ Info: Results processing took 0.001 seconds.
[ Info: Results processing took 0.001 seconds.
[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.014 seconds.
[ Info: Results processing took 0.001 seconds.
[ Info: Results processing took 0.0


Run 1038: Pset = 0.0, B_set = 5150.256
Run 1039: Pset = 0.0, B_set = 7021.223
Run 1040: Pset = 3571.2018, B_set = 4377.5


[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.014 seconds.
[ Info: Results processing took 0.001 seconds.
[ Info: Results processing took 0.001 seconds.
[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.009 seconds.
[ Info: Results processing took 0.007 seconds.
[ Info: Results processing took 0.008 seconds.
[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.016 seconds.
[ Info: Results processing took 0.002 seconds.
[ Info: Results processing took 0.002 seconds.
Excessive output truncated after 524303 bytes.

Run 1041: Pset = 8899.4517, B_set = -671.373


[ Info: Model built. Optimizing...
┌ Info: MPC solved with 
└   termination_status(m) = OPTIMAL::TerminationStatusCode = 1
[ Info: Solving took 0.017 seconds.
[ Info: Results processing took 0.0 seconds.
[ Info: Results processing took 0.0 seconds.
[ Info: Model built. Optimizing...


In [8]:
if save_results
    open(outputs_path * "/" * filename * ".json","w") do f
      JSON.print(f, dispatch_results)
    end
end

In [9]:
if !isempty(monthly_demand_cost)
    total_demand = sum(monthly_demand_cost)
else
    total_demand = 0
end

print("\nEnergy cost (\$): ", format(round(energy_cost_kwh, digits=3), commas=true))
print("\nDemand cost (\$): ", format(total_demand, commas=true))
print("\nTotal bill (\$): ", format(energy_cost_kwh + total_demand, commas=true))

grid_purchase = sum(dispatch_results["ElectricUtility"]["electric_to_load_series_kw"]) +
                sum(dispatch_results["ElectricUtility"]["electric_to_storage_series_kw"])
print("\n\nGrid purchases (kWh): ", format(round(grid_purchase, digits=3), commas=true))
# print("\n\nPercent RE: ", (sum(load) - grid_purchase)/sum(load)*100)

if stopping_ts == 8760
    pv_re = sum((pv_prod_factor * pv_kw) - 
            (dispatch_results["PV"]["electric_to_storage_series_kw"] * (1 - charge_efficiency*discharge_efficiency)) -
            (dispatch_results["PV"]["electric_curtailed_series_kw"]))

    print("\nCalculated percent RE: ", ((pv_re)/sum(load)*100))
end

battery_throughput = (sum(dispatch_results["PV"]["electric_to_storage_series_kw"]) + 
                  sum(dispatch_results["ElectricUtility"]["electric_to_storage_series_kw"])) * charge_efficiency

BESS_out = (sum(dispatch_results["ElectricStorage"]["storage_to_load_series_kw"])) / discharge_efficiency

print("\n\nBattery throughput (kWh in): ", format(round(battery_throughput, digits=3), commas=true))
print("\nBattery throughput (kWh out): ", format(round(BESS_out, digits=3), commas=true))


Energy cost ($): 1,531,417.145
Demand cost ($): 1,946,987.427669
Total bill ($): 3,478,404.572789

Grid purchases (kWh): 19,096,948.833
Calculated percent RE: 66.01862805837303

Battery throughput (kWh in): 11,684,987.112
Battery throughput (kWh out): 11,684,987.09

In [10]:
df = DataFrame()

# df[!, "pv_prod_factor"] = pv_prod_factor
df[!, "pv_to_load_series_kw"] = dispatch_results["PV"]["electric_to_load_series_kw"]
df[!, "pv_to_storage_series_kw"] = dispatch_results["PV"]["electric_to_storage_series_kw"]
df[!, "pv_to_grid_series_kw"] = dispatch_results["PV"]["electric_to_grid_series_kw"]
df[!, "pv_curtailed_production_series_kw"] = dispatch_results["PV"]["electric_curtailed_series_kw"]

df[!, "storage_to_load_series_kw"] = dispatch_results["ElectricStorage"]["storage_to_load_series_kw"]
df[!, "soc_series_fraction"] = dispatch_results["ElectricStorage"]["soc_series_fraction"]

df[!, "grid_to_load_series_kw"] = dispatch_results["ElectricUtility"]["electric_to_load_series_kw"]
df[!, "grid_to_battery_series_kw"] = dispatch_results["ElectricUtility"]["electric_to_storage_series_kw"]
df[!, "grid_emissions_tonnes"] = dispatch_results["ElectricUtility"]["emissions_series_tonnes_CO2"]

if stopping_ts == 8760
    df[!, "emissions_factor_lbs_per_kwh"] = emissions_lbs_per_kwh
end

df[!, "load_series_kw"] = dispatch_results["ElectricLoad"]["load_series_kw"]
df[!, "cost_per_timestep"] = cost_series

# first(df, 15)
# filename = "emissions_dispatch_results.csv"
if save_results
    CSV.write(outputs_path * "/" * filename * ".csv", df);
end

"outputs/aries_scenario1_short.csv"